<a href="https://colab.research.google.com/github/achillesposiedon/TaxiGameRL/blob/master/SpaceInvaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!pip install gym pyvirtualdisplay 
!apt-get install -y xvfb python-opengl ffmpeg 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.4).
0 upgraded, 0 newly installed, 0 to remove and 56 not upgraded.


In [3]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from collections import deque

from IPython import display as ipythondisplay

In [5]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

In [0]:
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
env = wrap_env(gym.make("SpaceInvaders-v0"))

In [9]:
observation = env.reset()

while True:
  
    env.render()
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action) 
   
        
    if done: 
      break;
            
env.close()
show_video()

In [10]:
#random action evaluation
rdrwrd=0
obs=env.reset()
env.render()

for i in range(100):
  action=env.action_space.sample()# pick random action from sample space
  obs,rewrd,done,info=env.step(action)
  rdrwrd+=rewrd
  env.render()

print(rdrwrd)
show_video()

0.0


In [0]:
def imgpreprocess(img):
  #convert to graysample and downsample 210X160 to 105X 80
  return np.mean(img[::2,::2], axis=2).astype(np.float32)/255.0 #2x downsample cheapest memory for uint8


In [0]:
stacksize=4
stacked= deque([np.zeros((105,80), dtype=np.float32)for i in range(stacksize)], maxlen=4)
  
def stack(stackeds,state,newepisode):
  img=imgpreprocess(state)
  
  if newepisode:
    stackeds=deque([np.zeros((105,80), dtype=np.float32) for i in range(stacksize)], maxlen=4)
  
    stackeds.append(img)
    stackeds.append(img)
    stackeds.append(img)
    stackeds.append(img)
  
    stacks=np.stack(stackeds, axis=2)

  else:
    stackeds.append(img)
    stacks=np.stack(stackeds, axis=2)

  return stacks

In [0]:
### MODEL HYPERPARAMETERS
statesize = [105, 80, 4]      # Our input is a stack of 4 frames hence 105X80x4 (Width, height, channels) 
actionsize = env.action_space.n # 8 possible actions
learningrate =  0.00025      # Alpha (aka learning rate)

### TRAINING HYPERPARAMETERS
total_episodes = 50            # Total episodes for training
max_steps = 50000              # Max possible steps in an episode
batch_size = 64                # Batch size

# Exploration parameters for epsilon greedy strategy
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.00001           # exponential decay rate for exploration prob

# Q learning hyperparameters
gamma = 0.9                    # Discounting rate

### MEMORY HYPERPARAMETERS
pretrain_length = batch_size   # Number of experiences stored in the Memory when initialized for the first time
memory_size = 1000000          # Number of experiences the Memory can keep

# MODIFY THIS TO FALSE IF YOU JUST WANT TO SEE THE TRAINED AGENT
training = False
episode_render = True

